# Main Aggregation Script

In [1]:
#Reloads the lab.py and crystals.py modules to update any changes (after saving)
#If a new method or object is created, autoreload doesn't work and the 
#kernel needs to be closed and halted after saving and making a 'checkpoint'
#in this notebook


%load_ext autoreload
%autoreload 2

In [2]:
import ipas
import matplotlib.pyplot as plt
import numpy as np
from sqlite3 import Connection as SQLite3Connection
from sqlalchemy import create_engine, event, select
from sqlalchemy.engine import Engine
from sqlalchemy.orm import sessionmaker
import base
from ipas import lab_ice_agg_SQL as lab
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
from dask import delayed
import cloudpickle as pickle
from multiprocessing import Pool
from functools import partial
import time

# from ipas import IceCrystal as crys
# from ipas import IceClusterAggAgg_SQL as clus
# from ipas import plots_phiarr as plts

In [16]:
cluster = SLURMCluster(
    queue='kratos',
    walltime='04-23:00:00',
    cores=1,
    memory='7168MiB', #1 GiB = 1,024 MiB
    processes=1)

cluster.scale(20)

/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/distributed/deploy/local.py:138: UserWarning: diagnostics_port has been deprecated. Please use `dashboard_address=` instead
  "diagnostics_port has been deprecated. "
/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [17]:
client = Client(cluster)

In [ ]:
client

In [5]:
phioarr=np.logspace(-2, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)
#phioarr = phioarr[1:]
lenphio = len(phioarr)

reqarr = [90, 100, 200, 300]
#reqarr = [50]
numaspectratios=len(phioarr)
ch_dist='gamma'         #anything other than gamma uses the characteristic from the best distribution pdf (lowest SSE)
nclusters = 300        #changes how many aggregates per aspect ratio to consider
ncrystals = 50       #number of monomers per aggregate 1
minor = 'depth'        #'minorxy' from fit ellipse or 'depth' to mimic IPAS in IDL
rand_orient = True    #randomly orient the seed crystal and new crystal: uses first random orientation
save_plots = False     #saves all histograms to path with # of aggs and minor/depth folder
file_ext = 'eps'


In [3]:
def main():
 
    '''
    for phio in phioarr:
        for r in reqarr:
            b1 = lab.collect_clusters(session, phio, r, nclusters=nclusters,
                                ncrystals=ncrystals, rand_orient=rand_orient, lodge=0)     
    '''
    
    notebook=4
    for r in reqarr:
        #b3 = []
        print('r = ',r)
        count = 1
        pool = Pool(processes=20) #pools are reusable
        parallel_clus=partial(lab.collect_clusters, notebook=notebook, r=r, nclusters=nclusters,\
                                ncrystals=ncrystals, \
                                rand_orient=rand_orient)

        start = time.time()
        output = pool.imap_unordered(parallel_clus, phioarr)
        for done in output:
            print("after %3.1fsec: count:%s"%(time.time()-start, count))
            count +=1

#         for res in output:
#             print("(after %3.1fsec)  mono phi:%.3f  count:%s"%(time.time()-start, res[0].mono_phi, count))
#             count += 1
        print('closing')
        pool.close()
        print('joining')
        pool.join()
        print('new r')
    
    '''

    output = []
    for r in reqarr:
        for phio in phioarr:
    
            print('eq. vol rad', r, phio)

            output.append(delayed(lab.collect_clusters)(phio, r, nclusters=nclusters,
                                                        ncrystals=ncrystals, rand_orient=rand_orient))
            
    
    print(output)
    print('computing...')
    b4 = client.compute(output) 
    b4 = client.gather(b4)
        
    '''

    


In [ ]:
if __name__ == '__main__':

    %time main() 
    
#     filename = 'instance_3radii_iceagg_allrand_50xtalstot_20phi_rall_300agg_4_lastmono'
#     filehandler = open(filename, 'wb')
#     %time pickle.dump(b4, filehandler)
#     filehandler.close()
#     print('finished, creating engine 4!')
    
#     engine = create_engine('sqlite:///IPAS_test_4_lastmono.sqlite')
#     event.listen(engine, 'connect', _set_sqlite_pragma)
    
#     base.Base.metadata.create_all(engine, checkfirst=True)
#     Session = sessionmaker(bind=engine)
#     session = Session()
    
#     try:
#         for r in b4:
#             for obj in r:
#                 session.add_all(obj)  # crystal id has been appended into cluster relationship
#                 session.commit()

#     except:
#         print('in except')
#         raise

#     session.close()  
#     print('DONE!!!!!')
    
    #packed = msgpack.writeResult('msgpack_test', b1)
    #msgpack.unpackb(packed, encoding='utf-8')
    
    #filename = 'instance_3radii_iceagg_allrand_100xtalstot_20phi_r10_1000agg'
    #filehandler = open(filename, 'wb')
    #%time pickle.dump(b1, filehandler)
    #filehandler.close()
    #print('finished!')
    
    #CHANGE LOOKUP TABLE FILES TO HAVE RANGE OF PHI AND R
    #Kratos = 28*4nodes = 112 -> 28 req so 4 aspect ratios at a time
    #would take 5 runs

r =  90
nclus 90 0.026366508987303583 0
nclus 90 5.455594781168514 0
nclus 90 3.359818286283781 0
nclus 90 0.04281332398719394 0
nclus 90 61.584821106602604 0
nclus 90 8.858667904100823 0
nclus 90 100.0 0
nclus 90 0.29763514416313175 0
nclus 90 0.4832930238571752 0
nclus 90 0.7847599703514611 0
nclus 90 1.2742749857031335 0
nclus 90 2.06913808111479 0
nclus 90 23.357214690901213 0
nclus 90 37.92690190732246 0
nclus 90 0.06951927961775606 0
nclus 90 0.18329807108324356 0
nclus 90 0.016237767391887217 0
nclus 90 0.11288378916846889 0
nclus 90 14.38449888287663 0
nclus 90 0.04281332398719394 50
nclus 90 0.18329807108324356 50
nclus 90 0.06951927961775606 50
nclus 90 0.4832930238571752 50
nclus 90 0.11288378916846889 50
nclus 90 0.29763514416313175 50
nclus 90 0.026366508987303583 50
nclus 90 0.7847599703514611 50
nclus 90 0.016237767391887217 50
nclus 90 1.2742749857031335 50
nclus 90 3.359818286283781 50
nclus 90 5.455594781168514 50
nclus 90 2.06913808111479 50
nclus 90 8.85866790410082

In [9]:
engine = create_engine('sqlite:///IPAS_test.sqlite')
event.listen(engine, 'connect', _set_sqlite_pragma)

base.Base.metadata.create_all(engine, checkfirst=True)
Session = sessionmaker(bind=engine)
session = Session()

try:
    for r in b4:
        for obj in r:
            session.add_all(obj)  # crystal id has been appended into cluster relationship
            session.commit()

except:
    print('in except')
    raise

session.close()  
print('DONE!!!!!')


DONE!!!!!


In [ ]:
file = open('instance_3radii_iceagg_allrand_50xtalstot_20phi_rall_300agg_4', 'rb')
b4 = pickle.load(file)

In [ ]:
print(b4.phio)

In [8]:
engine = create_engine('sqlite:///IPAS_4.sqlite', convert_unicode=True)
event.listen(engine, 'connect', _set_sqlite_pragma)

base.Base.metadata.create_all(engine, checkfirst=True)
Session = sessionmaker(bind=engine)
session = Session()

try:
    for obj in b4:
        session.add_all(obj)  # crystal id has been appended into cluster relationship
        session.commit()

except:
    print('in except')
    raise

session.close()  
print('DONE!!!!!')

DONE!!!!!


In [7]:
engine = create_engine('sqlite:///IPAS_4.sqlite')
base.Base.metadata.create_all(engine, checkfirst=True)
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
print('AGGREGATE PROPERTIES ==============')

query = session.query(ipas.IceCluster)#, crys.IceCrystal)#.filter(crys.IceCrystal.id == 10)

#query=session.query(clus.IceCluster).filter(crystals = 1).all()
for agg in query.all():
    print(agg.id, agg.ncrystals, agg.agg_phi, agg.cplx, agg.crystal[0].id,  agg.crystal[0].phi, agg.crystal[0].r)

print('MONOMER PROPERTIES ================')
query = session.query(ipas.IceCrystal)
for mono in query.all():
    print(mono.id, mono.phi, mono.r, mono.rand_orient, mono.aggs.ncrystals, mono.aggs.agg_phi)


AGGREGATE PROPERTIES ==============
1 2 1.6489504154672905 0.25442105050249547 1 0.01 10
2 3 1.562575822893241 0.24701350855129756 2 0.01 10
3 4 2.81944429601944 0.4977042977604404 3 0.01 10
4 5 3.089112332868638 0.5089931541697518 4 0.01 10
5 2 5.238371335315405 0.1996783895861054 5 0.01 10
6 3 1.8542128911720588 0.40915079528406395 6 0.01 10
7 4 1.8142084720339733 0.6370539949371135 7 0.01 10
8 5 2.2217740125189858 0.6318843264338923 8 0.01 10
9 2 3.2745435266159633 0.23467444139659427 9 0.01 10
10 3 3.1090110136192934 0.4442264008579038 10 0.01 10
11 4 2.507983510642043 0.4951875974187354 11 0.01 10
12 5 3.2004457920116676 0.5303297462307383 12 0.01 10
13 2 2.1655884711828612 0.31110183021070326 13 0.01 10
14 3 2.3577074681842562 0.5292101921995932 14 0.01 10
15 4 2.2863521500052175 0.6999598679309846 15 0.01 10
16 5 1.5911679864399213 0.725434991199331 16 0.01 10
17 2 1.4001403518955824 0.2972268197803417 17 0.01 10
18 3 1.7322369774724504 0.4790013777010492 18 0.01 10
19 4 2.15142

276 5 2.2969999002696557 0.6157362089552242 276 0.026366508987303583 10
277 2 2.1347288472797463 0.37915534503003845 277 0.026366508987303583 10
278 3 2.54370877833751 0.5216365747358169 278 0.026366508987303583 10
279 4 2.7447506089559317 0.5294496284673994 279 0.026366508987303583 10
280 5 3.361832641987276 0.5817320168899555 280 0.026366508987303583 10
281 2 3.5887884545918407 0.29007141458092867 281 0.026366508987303583 10
282 3 1.7193661370480342 0.2956921990267772 282 0.026366508987303583 10
283 4 1.8100426219130608 0.588821371181259 283 0.026366508987303583 10
284 5 1.5612573460513175 0.6861205594742261 284 0.026366508987303583 10
285 2 1.4434174845663037 0.19953468135684133 285 0.026366508987303583 10
286 3 1.3406151866831482 0.15308563816720785 286 0.026366508987303583 10
287 4 1.9321790716970613 0.19108361646712993 287 0.026366508987303583 10
288 5 2.677337459996111 0.38854082511339516 288 0.026366508987303583 10
289 2 2.411128706413776 0.2556022933934664 289 0.02636650898730

405 2 1.4680689731044447 0.20137597960617204 405 0.04281332398719394 10
406 3 1.7683293405001554 0.6494335885363933 406 0.04281332398719394 10
407 4 1.394687506037128 0.5985521462056689 407 0.04281332398719394 10
408 5 1.8913998797773908 0.6447771971160643 408 0.04281332398719394 10
409 2 1.9847813067647717 0.5776150841322119 409 0.04281332398719394 10
410 3 2.7608120777171328 0.5663119485010906 410 0.04281332398719394 10
411 4 3.1942601028414686 0.5758252892507474 411 0.04281332398719394 10
412 5 3.8866698183717734 0.6180957382469432 412 0.04281332398719394 10
413 2 1.4041654637853975 0.1772513110142529 413 0.04281332398719394 10
414 3 1.7805923759252147 0.28760549109308614 414 0.04281332398719394 10
415 4 1.5787014512738196 0.5058042586037921 415 0.04281332398719394 10
416 5 1.420853236480162 0.3756199336578023 416 0.04281332398719394 10
417 2 5.033339126509656 0.15873459425620906 417 0.04281332398719394 10
418 3 1.3673799254894614 0.19568102587418992 418 0.04281332398719394 10
419 4

528 5 1.858670154630517 0.6884302166842802 528 0.06951927961775606 10
529 2 1.737411655922122 0.2522473409086784 529 0.06951927961775606 10
530 3 2.4832070988507615 0.34757580911393915 530 0.06951927961775606 10
531 4 2.5108256207126223 0.40084728247107027 531 0.06951927961775606 10
532 5 2.182070357722999 0.5191348745294029 532 0.06951927961775606 10
533 2 1.497136940094808 0.2149677128726564 533 0.06951927961775606 10
534 3 1.539529223541246 0.24988537522067034 534 0.06951927961775606 10
535 4 1.9187726981652333 0.23262605714380089 535 0.06951927961775606 10
536 5 2.1570784566315657 0.5381999905474957 536 0.06951927961775606 10
537 2 1.657372330785276 0.2656647547691122 537 0.06951927961775606 10
538 3 2.0120621608974694 0.613748930688396 538 0.06951927961775606 10
539 4 2.9382065110067463 0.6925434246533051 539 0.06951927961775606 10
540 5 2.8559115407885605 0.7720444274282925 540 0.06951927961775606 10
541 2 2.2401953001694044 0.3572152416627769 541 0.06951927961775606 10
542 3 1.9

650 3 1.8179066048540113 0.5856890343467309 650 0.11288378916846889 10
651 4 2.029306262112836 0.554964055051003 651 0.11288378916846889 10
652 5 2.2510627578386835 0.5876334131733653 652 0.11288378916846889 10
653 2 1.9282781755378808 0.3072709202691919 653 0.11288378916846889 10
654 3 2.116834994541434 0.44094728591801025 654 0.11288378916846889 10
655 4 2.6045547571316705 0.5043524514391298 655 0.11288378916846889 10
656 5 2.94555752387543 0.626413356990708 656 0.11288378916846889 10
657 2 2.268820192943794 0.31501185056033 657 0.11288378916846889 10
658 3 2.429748041492475 0.39701076094896526 658 0.11288378916846889 10
659 4 2.0082436792053526 0.44321423729937554 659 0.11288378916846889 10
660 5 2.466874815811565 0.5608716472147728 660 0.11288378916846889 10
661 2 2.7124401346919838 0.5097970561789006 661 0.11288378916846889 10
662 3 2.873618300229236 0.42769940695372605 662 0.11288378916846889 10
663 4 3.733603617982314 0.47959203949266055 663 0.11288378916846889 10
664 5 2.829532

...

1353 2.06913808111479 10 True 2 0.35128611120656456
1354 2.06913808111479 10 True 3 0.4303995322269826
1355 2.06913808111479 10 True 4 0.4535923933169402
1356 2.06913808111479 10 True 5 0.41108519326906867
1357 2.06913808111479 10 True 2 0.4926523181883809
1358 2.06913808111479 10 True 3 0.39448468979140583
1359 2.06913808111479 10 True 4 0.43575089707989456
1360 2.06913808111479 10 True 5 0.4024287849263377
1361 2.06913808111479 10 True 2 0.7783594895796805
1362 2.06913808111479 10 True 3 0.5200586451706142
1363 2.06913808111479 10 True 4 0.3860546321515607
1364 2.06913808111479 10 True 5 0.33013156864236043
1365 2.06913808111479 10 True 2 0.4164623970859391
1366 2.06913808111479 10 True 3 0.3986613370540011
1367 2.06913808111479 10 True 4 0.42695724094981163
1368 2.06913808111479 10 True 5 0.3830962441609001
1369 2.06913808111479 10 True 2 0.3237914796607374
1370 2.06913808111479 10 True 3 0.23784585835092864
1371 2.06913808111479 10 True 4 0.30237416705484893
1372 2.06913808111479 1

1609 5.455594781168514 10 True 2 0.177635135774063
1610 5.455594781168514 10 True 3 0.5319638863569363
1611 5.455594781168514 10 True 4 0.5305229170566683
1612 5.455594781168514 10 True 5 0.5674430400274111
1613 5.455594781168514 10 True 2 0.29382764661866734
1614 5.455594781168514 10 True 3 0.6636272903846772
1615 5.455594781168514 10 True 4 0.6479771314876634
1616 5.455594781168514 10 True 5 0.4570979166581926
1617 5.455594781168514 10 True 2 0.22585583053297256
1618 5.455594781168514 10 True 3 0.27111005644512026
1619 5.455594781168514 10 True 4 0.5296100236689965
1620 5.455594781168514 10 True 5 0.5092108026866994
1621 5.455594781168514 10 True 2 0.1779229316864309
1622 5.455594781168514 10 True 3 0.15821630014285035
1623 5.455594781168514 10 True 4 0.19061090147104584
1624 5.455594781168514 10 True 5 0.23868620224813186
1625 5.455594781168514 10 True 2 0.15415099521301492
1626 5.455594781168514 10 True 3 0.19207056322861144
1627 5.455594781168514 10 True 4 0.29888661366342373
1628

1850 14.38449888287663 10 True 3 0.08358628267169507
1851 14.38449888287663 10 True 4 0.12294086823524035
1852 14.38449888287663 10 True 5 0.4075236599043218
1853 14.38449888287663 10 True 2 0.17055858789028688
1854 14.38449888287663 10 True 3 0.15551109255545456
1855 14.38449888287663 10 True 4 0.16482205066916733
1856 14.38449888287663 10 True 5 0.3728137568209695
1857 14.38449888287663 10 True 2 0.1147758293784752
1858 14.38449888287663 10 True 3 0.49890064461819944
1859 14.38449888287663 10 True 4 0.4743486765604544
1860 14.38449888287663 10 True 5 0.45558582118537294
1861 14.38449888287663 10 True 2 0.09974881373290048
1862 14.38449888287663 10 True 3 0.22225146848906924
1863 14.38449888287663 10 True 4 0.2617810453404433
1864 14.38449888287663 10 True 5 0.54011205533052
1865 14.38449888287663 10 True 2 0.07462743411733151
1866 14.38449888287663 10 True 3 0.5613329281169097
1867 14.38449888287663 10 True 4 0.4101067622231916
1868 14.38449888287663 10 True 5 0.44489270914591506
186

2053 37.92690190732246 10 True 2 0.06378946441197311
2054 37.92690190732246 10 True 3 0.6871802644614
2055 37.92690190732246 10 True 4 0.4170004448394335
2056 37.92690190732246 10 True 5 0.5006946179582608
2057 37.92690190732246 10 True 2 0.02726424709635045
2058 37.92690190732246 10 True 3 0.5503934121223844
2059 37.92690190732246 10 True 4 0.576876265255532
2060 37.92690190732246 10 True 5 0.5888919041662578
2061 37.92690190732246 10 True 2 0.06268651478832821
2062 37.92690190732246 10 True 3 0.29166523941134015
2063 37.92690190732246 10 True 4 0.24572478010902635
2064 37.92690190732246 10 True 5 0.2002127514387556
2065 37.92690190732246 10 True 2 0.04411960520774477
2066 37.92690190732246 10 True 3 0.6325737535944254
2067 37.92690190732246 10 True 4 0.7023662990722322
2068 37.92690190732246 10 True 5 0.6194096415894228
2069 37.92690190732246 10 True 2 0.03463028865145274
2070 37.92690190732246 10 True 3 0.24771291966603032
2071 37.92690190732246 10 True 4 0.4261912415739927
2072 37.

2280 61.584821106602604 10 True 5 0.18884791369941645
2281 100.0 10 True 2 0.010794635568842257
2282 100.0 10 True 3 0.13567799455752147
2283 100.0 10 True 4 0.17514569488157272
2284 100.0 10 True 5 0.25708238741000417
2285 100.0 10 True 2 0.018183251560052374
2286 100.0 10 True 3 0.10116582240059098
2287 100.0 10 True 4 0.21885920158981392
2288 100.0 10 True 5 0.18211583357112043
2289 100.0 10 True 2 0.01628452273558077
2290 100.0 10 True 3 0.12257815528039938
2291 100.0 10 True 4 0.09002563208414292
2292 100.0 10 True 5 0.31887175031798753
2293 100.0 10 True 2 0.02071990879134762
2294 100.0 10 True 3 0.4004603281591625
2295 100.0 10 True 4 0.5126048281089742
2296 100.0 10 True 5 0.44520261004745365
2297 100.0 10 True 2 0.016349993256299412
2298 100.0 10 True 3 0.20174601887799573
2299 100.0 10 True 4 0.28659131270855837
2300 100.0 10 True 5 0.5053123310348686
2301 100.0 10 True 2 0.015291818317207617
2302 100.0 10 True 3 0.22674327703440247
2303 100.0 10 True 4 0.2209836019171621
230

In [18]:
print('AGGREGATE PROPERTIES ==============')
query = session.query(ipas.IceCluster).filter(ipas.IceCluster.ncrystals == 3)

#query = session.query(clus.IceCluster)
for agg in query:
    print(agg.id, agg.ncrystals, agg.agg_phi, agg.agg_r, agg.id)

print('MONOMER PROPERTIES ================')
query = session.query(ipas.IceCrystal)
for mono in query.all():
    print(mono.id, mono.phi, mono.r, mono.rand_orient)

AGGREGATE PROPERTIES ==============
2 3 1.8740173820466801 59.37698841565121 2
6 3 2.3859356337384865 55.6787106938678 6
10 3 2.778599074182196 66.26468002911497 10
14 3 1.6747236078478691 62.99314501494472 14
18 3 1.6728977640739549 51.6818103407125 18
22 3 0.562322453228146 294.5122474931198 22
26 3 0.18439825476181357 211.27347388931824 26
30 3 0.36015275168692856 340.95968246474297 30
34 3 0.39139316984619915 244.6557059168472 34
38 3 0.2983764505124918 230.7597104241278 38
42 3 1.4532654870560684 60.4445862691194 42
46 3 1.7413433527872957 52.41042316482967 46
50 3 2.750634979565331 61.86103805993745 50
54 3 2.4786553214917566 70.60972644840311 54
58 3 2.705832162066122 65.44225636068938 58
62 3 0.13540536345156187 299.8724499109296 62
66 3 0.3313563839993581 318.3507886940626 66
70 3 0.4351117599762381 262.1617040476348 70
74 3 0.19861334526022234 250.1455844660929 74
78 3 0.16084564788548564 248.01778158792172 78
MONOMER PROPERTIES ================
1 0.01 9.999999999999996 True


/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as ipas.ice_crystal.IceCrystal, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
[autoreload of ipas.ice_crystal failed: Traceback (most recent call last):
  File "/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
 

In [ ]:
file = open('instance_3radii_iceagg_allrand_40xtalstot_20phi_r10', 'rb')
b1 = pickle.load(file)

In [ ]:
print(b1.rxs.shape)
print(np.max(b1.rxs))
print(np.max(b1.rys))
print(np.max(b1.rzs))

In [ ]:
#a=x b=y c=z

#dunnavan: a>= b >= c
#in our case z > y > x
phiba = b1.rxs/b1.rzs
phica = b1.rys/b1.rzs
#H, xedges, yedges = np.histogram2d(phica[10,:,2],phiba[10,:,2], normed=True)
#plt.imshow(H)
ax = sns.jointplot(x=phiba[1,:,2], y=phica[1,:,2], kind='hex')
ax = sns.jointplot(x=phiba[1,:,2], y=phica[1,:,2], kind='kde')
ax = sns.jointplot(x=phiba[19,:,2], y=phica[19,:,2], kind='hex', color='r')
ax = sns.jointplot(x=phiba[19,:,2], y=phica[19,:,2], kind='kde', color='r')



In [ ]:
n, bins, patches = plt.hist(phiba[10,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(phica[10,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)

In [ ]:
#axes rand plates
n, bins, patches = plt.hist(b1.rxs[1,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[1,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[1,:,2], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
#axes rand columns 
n, bins, patches = plt.hist(b1.rxs[19,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[19,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[19,:,2], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
n, bins, patches = plt.hist(b1.rxs[1,:,19], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[1,:,19], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[1,:,19], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
phios = [0,5,10,15,19]
print(b1.phio)
phiba = b1.rxs/b1.rzs
phica = b1.rys/b1.rzs
for i in phios:
    print(b1.phio[i])
    if i > 9:
        color = 'r'
    else:
        color = 'b'
    sns.jointplot(x=phiba[i,:,20], y=phica[i,:,20], kind='kde', xlim=[0,1], ylim=[0,1], color=color)
    

In [ ]:
file = open('instance_3radii_iceagg_allflat_40xtalstot_20phi_r10', 'rb')
b1 = pickle.load(file)

In [ ]:
phiba = b1.rxs/b1.rzs
phica = b1.rys/b1.rzs
phios = [0,5,10,15,19]
print(b1.phio)
for i in phios:
    print(b1.phio[i])
    if i > 9:
        color = 'r'
    else:
        color = 'b'
    sns.jointplot(x=phiba[i,:,38], y=phica[i,:,38], kind='kde', xlim=[0,1], ylim=[0,1], color=color)
    
    

In [ ]:
#a_ellipse (rzs) to a_mono (length for columns, width for plates)

if phio <= 1.0 #plates
    ellipse_ratio = mean(rzs[0:20:5,:,:])/mean(width[0:20:5,:,:])
else: #columns
    ellipse_ratio = mean(rzs[0:20:5,:,:])/mean(length[0:20:5,:,:])

plt.plot(np.arange(2,39,1), ellipse_ratio)


In [ ]:
filename = 'instance_3radii_aggagg_allflat_40xtalstot_20phi_r10'
filehandler = open(filename, 'wb')
#%time pickle.dump(b1, filehandler)
#filehandler.close()
#print('finished!')

In [ ]:
b5.ch_req_shape = b5.ch_req.reshape(28, 20, 40)[:,:,:-1]
b5.ch_majorax_shape = b5.ch_majorax.reshape(28, 20, 40)[:,:,:-1]
b5.ch_minorax_shape = b5.ch_minorax.reshape(28, 20, 40)[:,:,:-1]
b5.chphi_shape = b5.chphi.reshape(28, 20, 40)[:,:,:-1]
b5.ch_dd_shape = b5.ch_dd.reshape(28, 20, 40)[:,:,:-1] 

In [ ]:
phioarr=np.logspace(-2, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)
#phioarr = phioarr[1:]
reqarr = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000]
ncrystals=39

f1 = open('find_N_lookup_iceagg_flat_rall.dat',"w")
f2 = open('major_axis_lookup_iceagg_flat_rall.dat',"w")
f3 = open('minor_axis_lookup_iceagg_flat_rall.dat',"w")
f4 = open('dd_lookup_iceagg_flat_rall.dat',"w")

for i in range(len(reqarr)):
    for j in range(len(phioarr)):   
        for N in range((ncrystals)):
            #f1.write('%.3f\t %.1f\t %.4f\t %10.4f\t\n'%(b1.phioarr[j], reqarr[i], b1.chphi[N], b1.ch_req[N]))
            f1.write('{:8.4f} {:8.2f} {:8.4f} {:10.2f}\n'.format(b5.phioarr[j], reqarr[i], b5.chphi_shape[i,j,N], b5.ch_req_shape[i,j,N]))
            #f2.write('%.3f\t %.1f\t %.4f\t\n'%(b1.phioarr[j], reqarr[i], b1.ch_majorax[N]))
            f2.write('{:8.4f} {:8.2f} {:10.2f}\n'.format(b5.phioarr[j], reqarr[i], b5.ch_majorax_shape[i,j,N]))
            #print(b1.phioarr[j], reqarr[i], b1.ch_majorax[N])
           
            #minor_axis
            f3.write('{:8.4f} {:8.2f} {:8.4f}\n'.format(b5.phioarr[j], reqarr[i], b5.ch_minorax_shape[i,j,N]))
            #f3.write('%.3f\t %.1f\t %.4f\t\n'%(b1.phioarr[j], reqarr[i], b1.ch_minorax[N]))
            #density change
            f4.write("{:8.4f} {:8.2f} {:8.4f}\n".format(b5.phioarr[j], reqarr[i], b5.ch_dd_shape[i,j,N]))
            #f4.write('%5.3f\t %5.1f\t %.5f\t\n'%(b1.phioarr[j], reqarr[i], b1.ch_dd[N]))
            
f1.close()
f2.close()
f3.close()
f4.close()
